## This code performs whole-brain simulations where thermo- and hygrosensory neurons are activated and the activity of all other neurons is recorded.  This set of simulations will simulate specific cell types.

### Simulations are based on the leaky integrate and fire model by Shiu et al. (bioRxiv, 2023). 

### This notebook is adapted from one generously shared by Philip Shiu et al. (https://github.com/philshiu/Drosophila_brain_model)

In [6]:
from model import run_exp
from model import default_params as params
import utils as utl
from brian2 import Hz

config = {
    'path_res'  : './results/cellTypes',                     # directory to store results
    'path_comp' : './ShiuFiles/Completeness_783.csv',        # csv of the complete list of Flywire neurons
    'path_con'  : './ShiuFiles/Connectivity_783.parquet',    # connectivity data
    'n_proc'    : -1,                                               # number of CPU cores (-1: use all)
}

## Underlying connectivity data
The connectivity of the fly brain is stored in the folowing files:
- neurons present: `config['path_comp']`
- connectivity between neurons: `config['path_con]`

## Model parameters
The equation and constants for the leaky integrate and fire model are defined 
in the dictionary `default_params` in the beginning of the file `model.py`:

```
default_params = {
    # trials
    't_run'     : 1000 * ms,              # duration of trial
    'n_run'     : 30,                     # number of runs

    'v_0'       : -52 * mV,               # resting potential
    'v_rst'     : -52 * mV,               # reset potential after spike
    [...]
```
We can also change values
and pass the modified dictionary to the model (see Experiment 1).

## First, define each set of GRNs.
## We are using the same neurons as for the connectome analyses.

In [7]:
# VP2 thermosensory neurons reactive to heating
vp2 = [
    720575940619930534,
    720575940611720362,
    720575940615856345,
    720575940617041728,
    720575940646372228,
    720575940636878254,
    720575940638389437
    ]

# VP3a thermosensory neurons reactive to cooling
vp3a = [
    720575940616999069,
    720575940621659563,
    720575940621831804,
    720575940625293514,
    720575940627677258,
    720575940613191591,
    720575940650751873
    ]

# VP1m thermosensory? neurons, putative humid reactivity
vp1m = [
    720575940613794114,
    720575940616039197,
    720575940618512624,
    720575940619024774,
    720575940623793741,
    720575940627944464,
    720575940629007230,
    720575940631875298,
    720575940632913461,
    720575940633059381,
    720575940635969399,
    720575940646785924,
    720575940622082518
    ]

# VP4 hygrosensory neurons reactive to dryness
vp4 = [
    720575940613914774,
    720575940616239873,
    720575940617969961,
    720575940620602236,
    720575940620634537,
    720575940624673936,
    720575940630644281,
    720575940633280107,
    720575940638509952,
    720575940611377893,
    720575940614711638,
    720575940614919709,
    720575940618531544,
    720575940618574983,
    720575940619075073,
    720575940621009023,
    720575940622581651,
    720575940623030861,
    720575940623065013,
    720575940625868184,
    720575940626256992,
    720575940627150991,
    720575940628945516,
    720575940630282297,
    720575940637494837,
    720575940643707246,
    720575940622625097,
    720575940629275898,
    720575940633670904
    ]

# VP5 hygrosensory neurons reactive to humidity
vp5 = [
    720575940614634786,
    720575940635767524,
    720575940614842262,
    720575940614996018,
    720575940620459702,
    720575940622457028,
    720575940624011278,
    720575940626125626,
    720575940629693302,
    720575940632714465,
    720575940633274524,
    720575940637096026,
    720575940638300093,
    720575940648747385,
    720575940615553397,
    720575940627264262
    ]

# VP1d hygrosensory? neurons, putative reactive to evaporation
vp1d = [
    720575940603820716,
    720575940605264305,
    720575940609792206,
    720575940610460514,
    720575940614570793,
    720575940617665950,
    720575940617986089,
    720575940617986345,
    720575940620847086,
    720575940623476661,
    720575940627271230,
    720575940630271862,
    720575940635916942,
    720575940622112726,
    720575940623209703,
    720575940634781620
    ]

# VP1l hygrosensory? neurons, putative reactive to cooling
vp1l = [
    720575940614921501,
    720575940617987113,
    720575940620644446,
    720575940622700749,
    720575940651079158,
    720575940604931249,
    720575940613941533,
    720575940613942045,
    720575940613942301,
    720575940613942557,
    720575940625148174,
    720575940625657928,
    720575940626580361
    ]

In [8]:
flyid2name_vp2 = { f: 'vp2_{}'.format(i+1) for i, f in enumerate(vp2) }  # Heating
flyid2name_vp3a = { f: 'vp3a_{}'.format(i+1) for i, f in enumerate(vp3a) } # Cooling
flyid2name_vp1m = { f: 'vp1m_{}'.format(i+1) for i, f in enumerate(vp1m) } # Humidity?
flyid2name_vp4 = { f: 'vp4_{}'.format(i+1) for i, f in enumerate(vp4) } # Dryness
flyid2name_vp5 = { f: 'vp5_{}'.format(i+1) for i, f in enumerate(vp5) } # Humidity
flyid2name_vp1d = { f: 'vp1d_{}'.format(i+1) for i, f in enumerate(vp1d) } # Evaporation?
flyid2name_vp1l = { f: 'vp1l_{}'.format(i+1) for i, f in enumerate(vp1l) } # Cooling?

# view example
flyid2name_vp1l

{720575940614921501: 'vp1l_1',
 720575940617987113: 'vp1l_2',
 720575940620644446: 'vp1l_3',
 720575940622700749: 'vp1l_4',
 720575940651079158: 'vp1l_5',
 720575940604931249: 'vp1l_6',
 720575940613941533: 'vp1l_7',
 720575940613942045: 'vp1l_8',
 720575940613942301: 'vp1l_9',
 720575940613942557: 'vp1l_10',
 720575940625148174: 'vp1l_11',
 720575940625657928: 'vp1l_12',
 720575940626580361: 'vp1l_13'}

# Running simulations
## Background info:
To run a simulation exciting these nerons we have to call `run_exp` supplying the following:
- unique name for the simulation: `exp_name`
- a list of neurons we want to stimulate: `thermo_L`
- the connectivity data: `config['path_comp']` and `config['path_con]`
- path to store the output: `config['path_res']`
- number of CPU cores use: `config['n_procs]`

The `.parquet` file created during a simulation contains all spikes events of all neurons in the model.
We load the data again from disk by passing a list of result files to the `utl.load_exps` function.

The spike times can be converted to spike rates [Hz] via `utl.get_rate`, which requires the duration of each trial.
`utl.get_rate` returns `pandas.DataFrame` objects:
1. spike rate for each neuron (rows) in each experiment (column): `df_rate`
2. standard deviation of rate across trials: `df_rate_std`

For convenience, we can optionally pass the `flyid2name` dictionary to `utl.get_rate` in order to convert flywire IDs into
meaningful names.

In [9]:
#show default params
params

{'t_run': 1. * second,
 'n_run': 30,
 'v_0': -52. * mvolt,
 'v_rst': -52. * mvolt,
 'v_th': -45. * mvolt,
 't_mbr': 20. * msecond,
 'tau': 5. * msecond,
 't_rfc': 2.2 * msecond,
 't_dly': 1.8 * msecond,
 'w_syn': 275. * uvolt,
 'r_poi': 25. * hertz,
 'r_poi2': 0. * hertz,
 'f_poi': 250,
 'eqs': '\ndv/dt = (v_0 - v + g) / t_mbr : volt (unless refractory)\ndg/dt = -g / tau               : volt (unless refractory) \nrfc                            : second\n',
 'eq_th': 'v > v_th',
 'eq_rst': 'v = v_rst; w = 0; g = 0 * mV'}

## Thermosensory neuron cell types activation

In [10]:
# Run simulation for diff cell types
cell_types = [vp2, vp3a, vp1m,
              vp4, vp5, vp1d, vp1l]

labels = ['vp2','vp3a','vp1m',
              'vp4','vp5','vp1d','vp1l']

f2name = [flyid2name_vp2, flyid2name_vp3a, flyid2name_vp1m, 
          flyid2name_vp4, flyid2name_vp5, flyid2name_vp1d, flyid2name_vp1l]

# Run simulation at diff stim intensities
def stimulate_cellType(celltype, label, flyidname):
    for stim_rate in [5,10,15,20,25,50,75,100,125,150,175,200]: 
        
        prefix = label + '_' + str(stim_rate) + 'Hz'
        params['r_poi'] = stim_rate * Hz
        run_exp(exp_name=prefix, neu_exc=celltype, params=params, **config)
        
        # extract results
        datafilename = './results/cellTypes/' + prefix + '.parquet'
        df_spike = utl.load_exps([datafilename])
        df_rate, df_rate_std = utl.get_rate(df_spike, t_run=params['t_run'], n_run=params['n_run'], flyid2name=flyidname)
        
        # save dataframes to csv
        savepath = './results/cellTypes/CSVs/'
        df_rate.fillna(0).to_csv(savepath + prefix + '_rates.csv')
        df_rate_std.fillna(0).to_csv(savepath + prefix + '_std.csv')

for i in range(len(cell_types)):
    stimulate_cellType(cell_types[i], labels[i], f2name[i])

>>> Experiment:     vp2_25Hz
    Output file:    results\cellTypes\vp2_25Hz.parquet
    Excited neurons: 7
    Elapsed time:   1773 s
>>> Experiment:     vp2_50Hz
    Output file:    results\cellTypes\vp2_50Hz.parquet
    Excited neurons: 7
    Elapsed time:   1789 s
>>> Experiment:     vp2_75Hz
    Output file:    results\cellTypes\vp2_75Hz.parquet
    Excited neurons: 7


WARNING    c:\Users\snkyl\miniconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]


    Elapsed time:   1767 s
>>> Experiment:     vp2_100Hz
    Output file:    results\cellTypes\vp2_100Hz.parquet
    Excited neurons: 7


WARNING    c:\Users\snkyl\miniconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]


    Elapsed time:   1767 s
>>> Experiment:     vp2_125Hz
    Output file:    results\cellTypes\vp2_125Hz.parquet
    Excited neurons: 7


...

    Elapsed time:   1763 s
>>> Experiment:     vp1l_125Hz
    Output file:    results\cellTypes\vp1l_125Hz.parquet
    Excited neurons: 13


WARNING    c:\Users\snkyl\miniconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]


    Elapsed time:   1780 s
>>> Experiment:     vp1l_150Hz
    Output file:    results\cellTypes\vp1l_150Hz.parquet
    Excited neurons: 13


WARNING    c:\Users\snkyl\miniconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 [py.warnings]


    Elapsed time:   1788 s
>>> Experiment:     vp1l_175Hz
    Output file:    results\cellTypes\vp1l_175Hz.parquet
    Excited neurons: 13
    Elapsed time:   1790 s
>>> Experiment:     vp1l_200Hz
    Output file:    results\cellTypes\vp1l_200Hz.parquet
    Excited neurons: 13
    Elapsed time:   1798 s
